# Create images using label images and tables of desired values

In [ ]:
import sys
sys.path.insert(0, '../scripts')
import os as os
import os.path as op
import bish_utils as bu
def create_table_file(fname, header, label_value_pairs):
    '''
    Write a new file which starts with header and then writes
    in each line the label_value_pairs given as a list.
    '''
    with open(fname, 'w') as fil:
        fil.write(header)
        for pair in label_value_pairs:
            fil.write('%s\t%s\n' % (pair[0], pair[1]))
    return

def create_image(img_file, label_value_pairs, label_img, modify_img=None):
    '''
    Creates image by putting values present in label_value_pairs in 
    the selected labels in corresponding ROIs present in label_img.
    If modify_img is given update it instead of creating a new image
    with zero in all regions where the asked label is not present.
    '''
    import bish_utils as bu
    tbl_fl = 'tmp++table'
    header = 'labels\tnewValues\n'
    create_table_file(tbl_fl, header, label_value_pairs)
    cmd = ('../build/src/createImageFromLabelImage -t %s -l %s -o %s'
          % (tbl_fl, label_img, img_file))
    if modify_img:
        cmd += ' -m %s' % (modify_img)
    bu.print_and_execute(cmd)
    bu.print_and_execute('rm %s' % tbl_fl)
    

In [ ]:


in_img_dir = '/proj/asclepios/cluster/asclepios2/bkhanal/works/AdLemModel/results/patients/miriad_189_AD_M_10_MR_1'
#in_img_dir = '../results/patients/miriad_189_AD_M_10_MR_1'

csf_model_label = op.join(in_img_dir, 'maskwithCsfD2R0.nii.gz')
#aseg_label = op.join(in_img_dir, 'aseg_cropped.nii.gz')
aseg_label = op.join(in_img_dir, 'labelsWithCsfD2R0.nii.gz')

out_img_dir = in_img_dir
out_img = op.join(out_img_dir, 'muImageWmBstem10_GmCsf1.nii.gz')

# make all tissue and CSF 1 first
lab_val = [(1, 1), (2, 1)]
create_image(out_img, lab_val, csf_model_label)

# change labels of Brain stem and WM
h_val = 10
lab_val = [(16, h_val), (28, h_val), (60, h_val), (41, h_val), (2, h_val), (46, h_val), (7, h_val)]
create_image(out_img, lab_val, aseg_label, out_img)


In [ ]:
import os
import os.path as op
import bish_utils as bu

in_img_dir = '../results/patients/smallBrainTests/'
add_label_img = '../build/segGM_WM_CSF_FalxMedInria.nii.gz'
old_label_img = op.join(in_img_dir, 'segGM_CSF.nii.gz')

out_img_dir = in_img_dir
new_label_img = op.join(out_img_dir, 'segGM_CSF_Falx.nii.gz')
lab_val = [(4, 4)]
create_image(new_label_img, lab_val, add_label_img, old_label_img)

In [ ]:
import os
import os.path as op
import bish_utils as bu
tab_dir = '/proj/asclepios/cluster/asclepios2/mhadjham/Emotivity/Emotivity_All_N4/Simu_atrophy/'
out_dir = '/proj/asclepios/cluster/asclepios2/bkhanal/works/AdLemModel/results/patients/Patient_000068674859'
#out_dir = '/proj/asclepios/cluster/asclepios2/bkhanal/works/AdLemModel/results/patients/Patient_000098205180'

run_script = '/user/bkhanal/home/works/AdLemModel/scripts/create_atrophy_map.py'
seg = op.join(out_dir, 'labelsWithCsfForAtrophyD2R0_FS.nii.gz')
idx1, idx2 = [4], [3]
#idx1, idx2 = [3], [5, 6]
#idx1, idx2 = [4], [0, 1, 2, 3]
for i in idx1:
    for j in idx2:
        suffx = 'R%s_%s' % (i, j)
        tab_file = op.join(tab_dir, suffx+'.txt')
        atrophy_file = op.join(out_dir, 'atrophyMap'+suffx+'.nii.gz')
        cmd = ('python %s %s -seg_for_atrophy %s -atrophy_tables %s'
               % (run_script, atrophy_file, seg, tab_file))
        bu.print_and_execute(cmd)


## Create atrophy maps using segmentation images

In [ ]:
out_dir = '/proj/asclepios/cluster/asclepios2/bkhanal/works/AdLemModel/results/patients/Subject_0048'
seg = op.join(out_dir, 'labelsWithCsfForAtrophyD2R0_FS.nii.gz')

# a = 'atrophyMap_hippAmyg90p_crtx30p_wm6p.nii.gz'
# hipp_amyg, crtx, wm = -0.9, -0.3, -0.06 # -ve so that no need to invert the disp. field

a = 'atrophyMap_hippAmyg90p_crtx90p_wm40p.nii.gz'
hipp_amyg, crtx, wm = -0.9, -0.9, -0.4 # -ve so that no need to invert the disp. field


# FS labels: (L, R) : Hipp->(17, 53); Amyg->(18, 54); Crtx->(3, 42); WM->(2, 41)
a_val = [(17, hipp_amyg), (53, hipp_amyg), (18, hipp_amyg), (54, hipp_amyg),
         (3, crtx), (42, crtx), (2, wm), (41, wm)]


create_image(op.join(out_dir, a), a_val, seg)


## Modify existing atrophy maps

In [ ]:
import image_math as img_mth

a_map_opposite = '../results/patients/Subject_0048/Div_Mask_output_stationary_velocity_field_4_0048_2.0_0.15_3.0.nii.gz'
a_map = '../results/patients/Subject_0048/Minus_Div_Mask_output_stationary_velocity_field_4_0048_2.0_0.15_3.0.nii.gz'
#img_mth.mult_scalar_imgs([a_map_opposite], [a_map], -1)
a_map_zeroed_allcsf = '../results/patients/Subject_0048/atrophyMap_lcc_zeroedAllCsf.nii.gz'
a_map_zeroed_sulcalcsf = '../results/patients/Subject_0048/atrophyMap_lcc_zeroedOnlySulcalCsf.nii.gz'

seg_all_csf = '../results/patients/Subject_0048/segForModelD2R0_FS_allCSF.nii.gz'
seg_sulcal_csf = '../results/patients/Subject_0048/segForModelD2R0_FS_onlySulcalCSF.nii.gz'
label_vals = [(1, 0), (0, 0)]

#Use Script create_atrophy_map!! I need to crop the Jacobians, which is done
# over there. There are many more options over there and is much easier to
# use than do them again here. 